# Введение в простой RAG

Retrieval-Augmented Generation (RAG) - это гибридный подход, объединяющий информационный поиск с генеративными моделями. Он улучшает производительность языковых моделей за счет включения внешних знаний, что повышает точность и фактическую корректность.

В простой реализации RAG мы выполняем следующие шаги:

1. **Загрузка данных**: Загрузка и предварительная обработка текстовых данных.

2. **Разбиение на чанки**: Разделение данных на меньшие части для улучшения производительности поиска.

3. **Создание эмбеддингов**: Преобразование текстовых чанков в числовые представления с помощью модели эмбеддингов.

4. **Семантический поиск**: Поиск релевантных чанков на основе пользовательского запроса.

5. **Генерация ответа**: Использование языковой модели для генерации ответа на основе найденного текста.

Этот ноутбук реализует простой подход RAG, оценивает ответы модели и исследует различные улучшения.

## Настройка окружения

Начнем с импорта необходимых библиотек.

In [1]:
import fitz
import os
import numpy as np
import json
from openai import OpenAI

## Извлечение текста из PDF файла

Для реализации RAG нам сначала нужен источник текстовых данных. В данном случае мы извлекаем текст из PDF файла с помощью библиотеки PyMuPDF.

In [3]:
def extract_text_from_pdf(pdf_path):
    """
    Извлекает текст из PDF файла.

    Аргументы:
    pdf_path (str): Путь к PDF файлу.

    Возвращает:
    str: Извлеченный текст из PDF.
    """
    # Открываем PDF файл
    mypdf = fitz.open(pdf_path)
    all_text = ""  # Инициализируем пустую строку для хранения текста

    # Проходим по каждой странице PDF
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]  # Получаем страницу
        text = page.get_text("text")  # Извлекаем текст со страницы
        all_text += text  # Добавляем текст к общему содержимому

    return all_text  # Возвращаем извлеченный текст

## Разбиение извлеченного текста на чанки

После извлечения текста мы разделяем его на меньшие, перекрывающиеся части для повышения точности поиска.

In [4]:
def chunk_text(text, n, overlap):
    """
    Разбивает текст на части заданного размера с перекрытием.

    Аргументы:
    text (str): Текст для разбиения.
    n (int): Количество символов в каждом чанке.
    overlap (int): Количество перекрывающихся символов между чанками.

    Возвращает:
    List[str]: Список текстовых чанков.
    """
    chunks = []  # Инициализируем пустой список для чанков
    
    # Проходим по тексту с шагом (n - overlap)
    for i in range(0, len(text), n - overlap):
        # Добавляем чанк текста от i до i + n
        chunks.append(text[i:i + n])

    return chunks  # Возвращаем список чанков

## Настройка клиента OpenAI API

Инициализируем клиент OpenAI для генерации эмбеддингов и ответов.

In [ ]:
# Инициализируем клиент OpenAI с базовым URL и API ключом
client = OpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.getenv("OPENAI_API_KEY")  # Получаем API ключ из переменных окружения
)

## Извлечение и разбиение текста из PDF файла

Теперь мы загружаем PDF, извлекаем текст и разбиваем его на чанки.

In [ ]:
# Указываем путь к PDF файлу
pdf_path = "data/AI_Information.pdf"

# Извлекаем текст из PDF
extracted_text = extract_text_from_pdf(pdf_path)

# Разбиваем текст на чанки по 1000 символов с перекрытием 200 символов
text_chunks = chunk_text(extracted_text, 1000, 200)

# Выводим количество созданных чанков
print("Количество текстовых чанков:", len(text_chunks))

# Выводим первый чанк
print("\nПервый текстовый чанк:")
print(text_chunks[0])

## Создание эмбеддингов для текстовых чанков

Эмбеддинги преобразуют текст в числовые векторы, что позволяет эффективно выполнять поиск по сходству.

In [ ]:
def create_embeddings(text, model="BAAI/bge-en-icl"):
    """
    Создает эмбеддинги для заданного текста с использованием указанной модели.

    Аргументы:
    text (str): Входной текст для создания эмбеддингов.
    model (str): Модель для создания эмбеддингов. По умолчанию "BAAI/bge-en-icl".

    Возвращает:
    dict: Ответ API OpenAI, содержащий эмбеддинги.
    """
    # Создаем эмбеддинги для входного текста с использованием указанной модели
    response = client.embeddings.create(
        model=model,
        input=text
    )

    return response  # Возвращаем ответ с эмбеддингами

# Создаем эмбеддинги для текстовых чанков
response = create_embeddings(text_chunks)

## Семантический поиск

Реализуем косинусную схожесть для поиска наиболее релевантных текстовых чанков по запросу пользователя.

In [ ]:
def cosine_similarity(vec1, vec2):
    """
    Вычисляет косинусную схожесть между двумя векторами.

    Аргументы:
    vec1 (np.ndarray): Первый вектор.
    vec2 (np.ndarray): Второй вектор.

    Возвращает:
    float: Косинусная схожесть между векторами.
    """
    # Вычисляем скалярное произведение векторов и делим на произведение их норм
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def semantic_search(query, text_chunks, embeddings, k=5):
    """
    Выполняет семантический поиск по текстовым чанкам с использованием запроса и эмбеддингов.

    Аргументы:
    query (str): Запрос для семантического поиска.
    text_chunks (List[str]): Список текстовых чанков для поиска.
    embeddings (List[dict]): Список эмбеддингов для текстовых чанков.
    k (int): Количество наиболее релевантных чанков для возврата. По умолчанию 5.

    Возвращает:
    List[str]: Список k наиболее релевантных текстовых чанков.
    """
    # Создаем эмбеддинг для запроса
    query_embedding = create_embeddings(query).data[0].embedding
    similarity_scores = []  # Инициализируем список для хранения оценок схожести

    # Вычисляем оценки схожести между эмбеддингом запроса и каждым чанком
    for i, chunk_embedding in enumerate(embeddings):
        similarity_score = cosine_similarity(np.array(query_embedding), np.array(chunk_embedding.embedding))
        similarity_scores.append((i, similarity_score))  # Добавляем индекс и оценку

    # Сортируем оценки схожести по убыванию
    similarity_scores.sort(key=lambda x: x[1], reverse=True)
    # Получаем индексы k наиболее схожих чанков
    top_indices = [index for index, _ in similarity_scores[:k]]
    # Возвращаем k наиболее релевантных чанков
    return [text_chunks[index] for index in top_indices]

## Выполнение запроса по извлеченным чанкам

In [ ]:
# Загружаем валидационные данные из JSON файла
with open('data/val.json') as f:
    data = json.load(f)

# Извлекаем первый запрос из валидационных данных
query = data[0]['question']

# Выполняем семантический поиск для 2 наиболее релевантных чанков
top_chunks = semantic_search(query, text_chunks, response.data, k=2)

# Выводим запрос
print("Запрос:", query)

# Выводим 2 наиболее релевантных чанка
for i, chunk in enumerate(top_chunks):
    print(f"Контекст {i + 1}:\n{chunk}\n=====================================")

## Генерация ответа на основе найденных чанков

In [ ]:
# Определяем системный промт для AI ассистента
system_prompt = "Вы - AI ассистент, который отвечает строго на основе предоставленного контекста. Если ответ не может быть получен непосредственно из контекста, ответьте: 'У меня недостаточно информации для ответа на этот вопрос.'"

def generate_response(system_prompt, user_message, model="meta-llama/Llama-3.2-3B-Instruct"):
    """
    Генерирует ответ от AI модели на основе системного промта и пользовательского сообщения.

    Аргументы:
    system_prompt (str): Системный промт, определяющий поведение AI.
    user_message (str): Сообщение или запрос пользователя.
    model (str): Модель для генерации ответа. По умолчанию "meta-llama/Llama-3.2-3B-Instruct".

    Возвращает:
    dict: Ответ от AI модели.
    """
    response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]
    )
    return response

# Создаем пользовательский промт на основе найденных чанков
user_prompt = "\n".join([f"Контекст {i + 1}:\n{chunk}\n=====================================\n" for i, chunk in enumerate(top_chunks)])
user_prompt = f"{user_prompt}\nВопрос: {query}"

# Генерируем ответ AI
ai_response = generate_response(system_prompt, user_prompt)

## Оценка ответа AI

Сравниваем ответ AI с ожидаемым ответом и присваиваем оценку.

In [ ]:
# Определяем системный промт для системы оценки
evaluate_system_prompt = "Вы - интеллектуальная система оценки, задача которой - анализировать ответы AI ассистента. Если ответ AI очень близок к правильному ответу, присвойте оценку 1. Если ответ неверен или неудовлетворителен по сравнению с правильным ответом, присвойте оценку 0. Если ответ частично соответствует правильному ответу, присвойте оценку 0.5."

# Создаем промт для оценки, комбинируя запрос, ответ AI, правильный ответ и системный промт
evaluation_prompt = f"Запрос пользователя: {query}\nОтвет AI:\n{ai_response.choices[0].message.content}\nПравильный ответ: {data[0]['ideal_answer']}\n{evaluate_system_prompt}"

# Генерируем ответ оценки
evaluation_response = generate_response(evaluate_system_prompt, evaluation_prompt)

# Выводим результат оценки
print(evaluation_response.choices[0].message.content)